In [2]:
# imports
import pyomo.environ as pe
import pyomo.opt as po
import math

# Inputs

In [ ]:
# -------------------------------------------------------------------------
# SETS
# -------------------------------------------------------------------------
factories = ['IJmuiden', 'Segal', 'South Wales']
customer_areas = ['Bochum', 'Boenen', 'Dortmund', 'Gelsenkirchen', 'Hagen', 'Iserlohn', 'Neuss', 'Schwerte']
long_bar_types = ['1', '2']
rebar_types = ['A', 'B', 'C']
numOfPeriods = 4
# vehicles = ['V_IJ', 'V_SE', 'V_SW']


# Parameters needed for steel capacity/fleet capacity calculations
# length per long bar type
length_lb = {'1': 9, '2': 12}

# length per rebar type
length_rb = {'A': 2.4, 'B': 3.6, 'C': 4.2}

# diameter for rebar and long bar types
diameter = {'1': 0.057, '2': 0.057, 'A': 0.057, 'B': 0.057, 'C': 0.057}

# density of steel in t/m3
density = 7.85

# ------------------------------------------------------------------------
# Steel and Vehicle capactity
# -------------------------------------------------------------------------
# Steel capacity per factory per period in tons
steel_capacity = {'IJmuiden': 12, 'Segal': 10, 'South Wales': 28}
vehicle_capacity = 10  # in tons


# -------------------------------------------------------------------------
# Transportation costs
# -------------------------------------------------------------------------
# Fixed cost per fleet per period per factory
f_fleet = {'IJmuiden': 130, 'Segal': 150, 'South Wales': 100}

# variable costs per km 
c_fleet = {'IJmuiden': 3, 'Segal': 3, 'South Wales': 3}

# -------------------------------------------------------------------------
# Demand 
# -------------------------------------------------------------------------

# demand for each rebar type per period per customer area
demand = {
    # Rebar A
    ('A', 1, 'Bochum'): 2,  ('A', 1, 'Boenen'): 4,  ('A', 1, 'Dortmund'): 2,  ('A', 1, 'Gelsenkirchen'): 5,
    ('A', 1, 'Hagen'): 19,  ('A', 1, 'Iserlohn'): 13, ('A', 1, 'Neuss'): 20, ('A', 1, 'Schwerte'): 4,

    ('A', 2, 'Bochum'): 6,  ('A', 2, 'Boenen'): 8,  ('A', 2, 'Dortmund'): 7,  ('A', 2, 'Gelsenkirchen'): 5,
    ('A', 2, 'Hagen'): 23,  ('A', 2, 'Iserlohn'): 19, ('A', 2, 'Neuss'): 16, ('A', 2, 'Schwerte'): 5,

    ('A', 3, 'Bochum'): 5,  ('A', 3, 'Boenen'): 5,  ('A', 3, 'Dortmund'): 6,  ('A', 3, 'Gelsenkirchen'): 5,
    ('A', 3, 'Hagen'): 25,  ('A', 3, 'Iserlohn'): 17, ('A', 3, 'Neuss'): 14, ('A', 3, 'Schwerte'): 3,

    ('A', 4, 'Bochum'): 3,  ('A', 4, 'Boenen'): 10, ('A', 4, 'Dortmund'): 5,  ('A', 4, 'Gelsenkirchen'): 5,
    ('A', 4, 'Hagen'): 16,  ('A', 4, 'Iserlohn'): 14, ('A', 4, 'Neuss'): 26, ('A', 4, 'Schwerte'): 4,

    # Rebar B
    ('B', 1, 'Bochum'): 4,  ('B', 1, 'Boenen'): 5,  ('B', 1, 'Dortmund'): 4,  ('B', 1, 'Gelsenkirchen'): 9,
    ('B', 1, 'Hagen'): 15,  ('B', 1, 'Iserlohn'): 22, ('B', 1, 'Neuss'): 12, ('B', 1, 'Schwerte'): 2,

    ('B', 2, 'Bochum'): 5,  ('B', 2, 'Boenen'): 8,  ('B', 2, 'Dortmund'): 5,  ('B', 2, 'Gelsenkirchen'): 10,
    ('B', 2, 'Hagen'): 33,  ('B', 2, 'Iserlohn'): 26, ('B', 2, 'Neuss'): 23, ('B', 2, 'Schwerte'): 8,

    ('B', 3, 'Bochum'): 7,  ('B', 3, 'Boenen'): 12, ('B', 3, 'Dortmund'): 8,  ('B', 3, 'Gelsenkirchen'): 6,
    ('B', 3, 'Hagen'): 31,  ('B', 3, 'Iserlohn'): 20, ('B', 3, 'Neuss'): 30, ('B', 3, 'Schwerte'): 2,

    ('B', 4, 'Bochum'): 8,  ('B', 4, 'Boenen'): 13, ('B', 4, 'Dortmund'): 10, ('B', 4, 'Gelsenkirchen'): 6,
    ('B', 4, 'Hagen'): 33,  ('B', 4, 'Iserlohn'): 27, ('B', 4, 'Neuss'): 30, ('B', 4, 'Schwerte'): 6,

    # Rebar C
    ('C', 1, 'Bochum'): 6,  ('C', 1, 'Boenen'): 6,  ('C', 1, 'Dortmund'): 7,  ('C', 1, 'Gelsenkirchen'): 10,
    ('C', 1, 'Hagen'): 12,  ('C', 1, 'Iserlohn'): 14, ('C', 1, 'Neuss'): 22, ('C', 1, 'Schwerte'): 5,

    ('C', 2, 'Bochum'): 7,  ('C', 2, 'Boenen'): 10, ('C', 2, 'Dortmund'): 6,  ('C', 2, 'Gelsenkirchen'): 9,
    ('C', 2, 'Hagen'): 35,  ('C', 2, 'Iserlohn'): 25, ('C', 2, 'Neuss'): 32, ('C', 2, 'Schwerte'): 6,

    ('C', 3, 'Bochum'): 7,  ('C', 3, 'Boenen'): 15, ('C', 3, 'Dortmund'): 4,  ('C', 3, 'Gelsenkirchen'): 9,
    ('C', 3, 'Hagen'): 33,  ('C', 3, 'Iserlohn'): 23, ('C', 3, 'Neuss'): 31, ('C', 3, 'Schwerte'): 7,

    ('C', 4, 'Bochum'): 7,  ('C', 4, 'Boenen'): 12, ('C', 4, 'Dortmund'): 12, ('C', 4, 'Gelsenkirchen'): 10,
    ('C', 4, 'Hagen'): 38,  ('C', 4, 'Iserlohn'): 24, ('C', 4, 'Neuss'): 31, ('C', 4, 'Schwerte'): 2,
}

# -------------------------------------------------------------------------
# Distance between factories and customer areas in km
# -------------------------------------------------------------------------
dist = {

    # From IJmuiden
    ('IJmuiden', 'IJmuiden'): 0,
    ('IJmuiden', 'Segal'): 284,
    ('IJmuiden', 'South Wales'): 826,
    ('IJmuiden', 'Bochum'): 250,
    ('IJmuiden', 'Boenen'): 282,
    ('IJmuiden', 'Dortmund'): 266,
    ('IJmuiden', 'Gelsenkirchen'): 234,
    ('IJmuiden', 'Hagen'): 289,
    ('IJmuiden', 'Iserlohn'): 299,
    ('IJmuiden', 'Neuss'): 259,
    ('IJmuiden', 'Schwerte'): 279,

    # From Segal
    ('Segal', 'IJmuiden'): 284,
    ('Segal', 'Segal'): 0,
    ('Segal', 'South Wales'): 750,
    ('Segal', 'Bochum'): 203,
    ('Segal', 'Boenen'): 242,
    ('Segal', 'Dortmund'): 222,
    ('Segal', 'Gelsenkirchen'): 198,
    ('Segal', 'Hagen'): 206,
    ('Segal', 'Iserlohn'): 226,
    ('Segal', 'Neuss'): 140,
    ('Segal', 'Schwerte'): 216,

    # From South Wales
    ('South Wales', 'IJmuiden'): 826,
    ('South Wales', 'Segal'): 750,
    ('South Wales', 'South Wales'): 0,
    ('South Wales', 'Bochum'): 866,
    ('South Wales', 'Boenen'): 914,
    ('South Wales', 'Dortmund'): 885,
    ('South Wales', 'Gelsenkirchen'): 859,
    ('South Wales', 'Hagen'): 903,
    ('South Wales', 'Iserlohn'): 913,
    ('South Wales', 'Neuss'): 843,
    ('South Wales', 'Schwerte'): 901,

    # From Bochum
    ('Bochum', 'IJmuiden'): 250,
    ('Bochum', 'Segal'): 203,
    ('Bochum', 'South Wales'): 866,
    ('Bochum', 'Bochum'): 0,
    ('Bochum', 'Boenen'): 55,
    ('Bochum', 'Dortmund'): 21,
    ('Bochum', 'Gelsenkirchen'): 19,
    ('Bochum', 'Hagen'): 41,
    ('Bochum', 'Iserlohn'): 51,
    ('Bochum', 'Neuss'): 56,
    ('Bochum', 'Schwerte'): 39,

    # From Boenen
    ('Boenen', 'IJmuiden'): 282,
    ('Boenen', 'Segal'): 242,
    ('Boenen', 'South Wales'): 914,
    ('Boenen', 'Bochum'): 55,
    ('Boenen', 'Boenen'): 0,
    ('Boenen', 'Dortmund'): 34,
    ('Boenen', 'Gelsenkirchen'): 56,
    ('Boenen', 'Hagen'): 44,
    ('Boenen', 'Iserlohn'): 54,
    ('Boenen', 'Neuss'): 102,
    ('Boenen', 'Schwerte'): 32,

    # From Dortmund
    ('Dortmund', 'IJmuiden'): 266,
    ('Dortmund', 'Segal'): 222,
    ('Dortmund', 'South Wales'): 885,
    ('Dortmund', 'Bochum'): 21,
    ('Dortmund', 'Boenen'): 34,
    ('Dortmund', 'Dortmund'): 0,
    ('Dortmund', 'Gelsenkirchen'): 34,
    ('Dortmund', 'Hagen'): 21,
    ('Dortmund', 'Iserlohn'): 36,
    ('Dortmund', 'Neuss'): 75,
    ('Dortmund', 'Schwerte'): 15,

    # From Gelsenkirchen
    ('Gelsenkirchen', 'IJmuiden'): 234,
    ('Gelsenkirchen', 'Segal'): 198,
    ('Gelsenkirchen', 'South Wales'): 859,
    ('Gelsenkirchen', 'Bochum'): 19,
    ('Gelsenkirchen', 'Boenen'): 56,
    ('Gelsenkirchen', 'Dortmund'): 34,
    ('Gelsenkirchen', 'Gelsenkirchen'): 0,
    ('Gelsenkirchen', 'Hagen'): 56,
    ('Gelsenkirchen', 'Iserlohn'): 66,
    ('Gelsenkirchen', 'Neuss'): 62,
    ('Gelsenkirchen', 'Schwerte'): 54,

    # From Hagen
    ('Hagen', 'IJmuiden'): 289,
    ('Hagen', 'Segal'): 206,
    ('Hagen', 'South Wales'): 903,
    ('Hagen', 'Bochum'): 41,
    ('Hagen', 'Boenen'): 44,
    ('Hagen', 'Dortmund'): 21,
    ('Hagen', 'Gelsenkirchen'): 56,
    ('Hagen', 'Hagen'): 0,
    ('Hagen', 'Iserlohn'): 20,
    ('Hagen', 'Neuss'): 66,
    ('Hagen', 'Schwerte'): 14,

    # From Iserlohn
    ('Iserlohn', 'IJmuiden'): 299,
    ('Iserlohn', 'Segal'): 226,
    ('Iserlohn', 'South Wales'): 913,
    ('Iserlohn', 'Bochum'): 51,
    ('Iserlohn', 'Boenen'): 54,
    ('Iserlohn', 'Dortmund'): 36,
    ('Iserlohn', 'Gelsenkirchen'): 66,
    ('Iserlohn', 'Hagen'): 20,
    ('Iserlohn', 'Iserlohn'): 0,
    ('Iserlohn', 'Neuss'): 85,
    ('Iserlohn', 'Schwerte'): 15,

    # From Neuss
    ('Neuss', 'IJmuiden'): 259,
    ('Neuss', 'Segal'): 140,
    ('Neuss', 'South Wales'): 843,
    ('Neuss', 'Bochum'): 56,
    ('Neuss', 'Boenen'): 102,
    ('Neuss', 'Dortmund'): 75,
    ('Neuss', 'Gelsenkirchen'): 62,
    ('Neuss', 'Hagen'): 66,
    ('Neuss', 'Iserlohn'): 85,
    ('Neuss', 'Neuss'): 0,
    ('Neuss', 'Schwerte'): 75,

    # From Schwerte
    ('Schwerte', 'IJmuiden'): 279,
    ('Schwerte', 'Segal'): 216,
    ('Schwerte', 'South Wales'): 901,
    ('Schwerte', 'Bochum'): 39,
    ('Schwerte', 'Boenen'): 32,
    ('Schwerte', 'Dortmund'): 15,
    ('Schwerte', 'Gelsenkirchen'): 54,
    ('Schwerte', 'Hagen'): 14,
    ('Schwerte', 'Iserlohn'): 15,
    ('Schwerte', 'Neuss'): 75,
    ('Schwerte', 'Schwerte'): 0,
}


In [4]:
# Calculating weights and Generating patterns
radius = diameter['A'] / 2  # 0.0285 m
weights_lb = {l: math.pi * (radius**2) * length_lb[l] * density for l in long_bar_types}
weights_rb = {r: math.pi * (radius**2) * length_rb[r] * density for r in rebar_types}

customer_weight = {}
for c in customer_areas:
    for t in range(1, numOfPeriods + 1):
        # Assumes demand dict is populated
        customer_weight[c, t] = sum(demand[r, t, c] * weights_rb[r] for r in rebar_types)

def generate_patterns(long_length, rebar_lengths):
    patterns = []
    max_A = int(long_length // rebar_lengths['A'])
    max_B = int(long_length // rebar_lengths['B'])
    max_C = int(long_length // rebar_lengths['C'])
    for a in range(max_A + 1):
        for b in range(max_B + 1):
            for c in range(max_C + 1):
                total_len = a*rebar_lengths['A'] + b*rebar_lengths['B'] + c*rebar_lengths['C']
                if 0 < total_len <= long_length:
                    patterns.append({'A': a, 'B': b, 'C': c})
    return patterns

patterns_1 = generate_patterns(length_lb['1'], length_rb)
patterns_2 = generate_patterns(length_lb['2'], length_rb)

# Sets en Parameters

In [6]:
model = pe.ConcreteModel(name="Production_Delivery_Planning")

# Sets
model.F = pe.Set(initialize=factories) # set of factories
model.C = pe.Set(initialize=customer_areas) # set of customers
model.N = pe.Set(initialize=factories + customer_areas) # set of factories + customers (all nodes in the network)
model.T = pe.Set(initialize=range(1, numOfPeriods + 1)) # set of periods (1, numOfperiods + 1 since it otherwise would be 0,1,2,3)
model.L = pe.Set(initialize=long_bar_types) # set of long bar types
model.R = pe.Set(initialize=rebar_types) # set of rebar types
model.P1 = pe.Set(initialize=range(len(patterns_1))) # set of cutting patterns for long bar type 1
model.P2 = pe.Set(initialize=range(len(patterns_2))) # set of cutting patterns for long bar type 2



# Decision Variables

In [13]:
# Variables
model.x1 = pe.Var(model.F, model.P1, model.T, domain=pe.NonNegativeIntegers) # non-negative integer of cutting patterns from longbar type 1, produced by factory f in period t
model.x2 = pe.Var(model.F, model.P2, model.T, domain=pe.NonNegativeIntegers) # non-negative interger of cutting patterns from longbar type 2, produced by factory f in period t
model.y = pe.Var(model.F, model.C, model.T, domain=pe.Binary) # binary indicator, is customer c served by factory f in period t
model.z = pe.Var(model.F, model.N, model.N, model.T, domain=pe.Binary) # binary indicator, is arc N,N used during the tour from factory f in period t
model.w = pe.Var(model.F, model.T, domain=pe.Binary) # binary indicator, is factory f used for production and serving in period t
model.u = pe.Var(model.F, model.C, model.T, domain=pe.NonNegativeReals) # MTZ order variable for subtour elimination (visit position of customer c)

# Parameters
# --- Params ---
model.dist = pe.Param(model.N, model.N, initialize=dist)

model.demand = pe.Param(model.R, model.T, model.C, initialize=lambda m, r, t, c: demand.get((r, t, c), 0))

model.steel_capacity = pe.Param(model.F, initialize=steel_capacity)

model.vehicle_capacity = pe.Param(initialize=vehicle_capacity)

model.f_fleet = pe.Param(model.F, initialize=f_fleet, within=pe.NonNegativeReals)
model.c_fleet = pe.Param(model.F, initialize=c_fleet, within=pe.NonNegativeReals)

model.weights_lb = pe.Param(model.L, initialize=weights_lb)

model.customer_weight = pe.Param(model.C, model.T, initialize=customer_weight)

# # Objective
# def obj_rule(model):
#     routing_cost = sum(model.z[f, i, j, t] * dist[i, j] * c_fleet[f] 
#                        for f in model.F for i in model.N for j in model.N for t in model.T if i != j)
#     fixed_cost = sum(model.w[f, t] * f_fleet[f] for f in model.F for t in model.T)
#     return routing_cost + fixed_cost
# model.Obj = pe.Objective(rule=obj_rule, sense=pe.minimize)

def obj_rule(model):
    routing_cost = sum(model.z[f, i, j, t] * model.dist[i, j] * model.c_fleet[f]
                       for f in model.F for i in model.N for j in model.N for t in model.T
                       if i != j)
    fixed_cost = sum(model.w[f, t] * model.f_fleet[f] for f in model.F for t in model.T)
    return routing_cost + fixed_cost

model.Obj = pe.Objective(rule=obj_rule, sense=pe.minimize)


'pyomo.core.base.var.IndexedVar'>) on block Production_Delivery_Planning with
a new Component (type=<class 'pyomo.core.base.var.IndexedVar'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.var.IndexedVar'>) on block Production_Delivery_Planning with
a new Component (type=<class 'pyomo.core.base.var.IndexedVar'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.var.IndexedVar'>) on block Production_Delivery_Planning with
a new Component (type=<class 'pyomo.core.base.var.IndexedVar'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.var.IndexedVar'>) on block Production_Delivery_Planning with
a new Component (type=<class 'pyomo.core.base.var.IndexedVar'>). This is
usually indicative of a modelling error. To a

# Constraints

In [28]:
# Constraints
def assign_rule(model, c, t):
    return sum(model.y[f, c, t] for f in model.F) == 1
model.AssignConstraint = pe.Constraint(model.C, model.T, rule=assign_rule)

def link_prod_rule(model, f, r, t):
    prod1 = sum(patterns_1[p][r] * model.x1[f, p, t] for p in model.P1)
    prod2 = sum(patterns_2[p][r] * model.x2[f, p, t] for p in model.P2)
    demand_assigned = sum(model.demand[r, t, c] * model.y[f, c, t] for c in model.C)
    return prod1 + prod2 >= demand_assigned

model.LinkProdConstraint = pe.Constraint(model.F, model.R, model.T, rule=link_prod_rule)

# def link_prod_rule(model, f, r, t):
#     prod1 = sum(patterns_1[p][r] * model.x1[f, p, t] for p in model.P1)
#     prod2 = sum(patterns_2[p][r] * model.x2[f, p, t] for p in model.P2)
#     demand_assigned = sum(demand[r, t, c] * model.y[f, c, t] for c in model.C)
#     return prod1 + prod2 >= demand_assigned
# model.LinkProdConstraint = pe.Constraint(model.F, model.R, model.T, rule=link_prod_rule)

def factory_cap_rule(model, f, t):
    weight_1 = sum(model.x1[f, p, t] * model.weights_lb['1'] for p in model.P1)
    weight_2 = sum(model.x2[f, p, t] * model.weights_lb['2'] for p in model.P2)
    return weight_1 + weight_2 <= model.steel_capacity[f]
model.FactoryCapConstraint = pe.Constraint(model.F, model.T, rule=factory_cap_rule)

# def factory_cap_rule(model, f, t):
#     weight_1 = sum(model.x1[f, p, t] * weights_lb['1'] for p in model.P1)
#     weight_2 = sum(model.x2[f, p, t] * weights_lb['2'] for p in model.P2)
#     return weight_1 + weight_2 <= steel_capacity[f]
# model.FactoryCapConstraint = pe.Constraint(model.F, model.T, rule=factory_cap_rule)

def vehicle_cap_rule(model, f, t):
    assigned_weight = sum(model.customer_weight[c, t] * model.y[f, c, t] for c in model.C)
    return assigned_weight <= model.vehicle_capacity * model.w[f, t]
model.VehicleCapConstraint = pe.Constraint(model.F, model.T, rule=vehicle_cap_rule)

# def vehicle_cap_rule(model, f, t):
#     assigned_weight = sum(customer_weight[c, t] * model.y[f, c, t] for c in model.C)
#     return assigned_weight <= vehicle_capacity * model.w[f, t]
# model.VehicleCapConstraint = pe.Constraint(model.F, model.T, rule=vehicle_cap_rule)


def leave_factory_rule(model, f, t):
    return sum(model.z[f, f, j, t] for j in model.C) == model.w[f, t]
model.LeaveFactoryConstraint = pe.Constraint(model.F, model.T, rule=leave_factory_rule)

def return_factory_rule(model, f, t):
    return sum(model.z[f, i, f, t] for i in model.C) == model.w[f, t]
model.ReturnFactoryConstraint = pe.Constraint(model.F, model.T, rule=return_factory_rule)

def flow_in_rule(model, f, c, t):
    return sum(model.z[f, i, c, t] for i in model.N if i != c) == model.y[f, c, t]
model.FlowInConstraint = pe.Constraint(model.F, model.C, model.T, rule=flow_in_rule)

def flow_out_rule(model, f, c, t):
    return sum(model.z[f, c, j, t] for j in model.N if j != c) == model.y[f, c, t]
model.FlowOutConstraint = pe.Constraint(model.F, model.C, model.T, rule=flow_out_rule)

def no_cross_factory_rule(model, f, i, j, t):
    if i in factories and j in factories:
        return model.z[f, i, j, t] == 0
    elif (i in factories and i != f) or (j in factories and j != f):
        return model.z[f, i, j, t] == 0
    return pe.Constraint.Skip
model.NoCrossFactoryConstraint = pe.Constraint(model.F, model.N, model.N, model.T, rule=no_cross_factory_rule)


def mtz_rule(model, f, i, j, t):
    if i != j:
        return model.u[f, i, t] - model.u[f, j, t] + len(model.C) * model.z[f, i, j, t] <= len(model.C) - 1
    return pe.Constraint.Skip
model.MTZConstraint = pe.Constraint(model.F, model.C, model.C, model.T, rule=mtz_rule)

# Extra constraints for only producing when a factory is used
# Precompute tight "M" values (max number of bars possible given steel capacity)
M1 = {f: math.floor(steel_capacity[f] / weights_lb['1']) for f in factories}  # for 9m bars
M2 = {f: math.floor(steel_capacity[f] / weights_lb['2']) for f in factories}  # for 12m bars

def prod_only_if_active_1_rule(model, f, t):
    return sum(model.x1[f, p, t] for p in model.P1) <= M1[f] * model.w[f, t]
model.ProdOnlyIfActive1 = pe.Constraint(model.F, model.T, rule=prod_only_if_active_1_rule)

def prod_only_if_active_2_rule(model, f, t):
    return sum(model.x2[f, p, t] for p in model.P2) <= M2[f] * model.w[f, t]
model.ProdOnlyIfActive2 = pe.Constraint(model.F, model.T, rule=prod_only_if_active_2_rule)



(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
Production_Delivery_Planning with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
Production_Delivery_Planning with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
Production_Delivery_Planning with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
Pr

# Solve and Print Results

In [29]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee=False, options={'TimeLimit': 3600})

print(f"\nOptimization Status: {result.solver.status}")
if result.solver.status == po.SolverStatus.ok:
    print(f"Total Objective Cost: €{pe.value(model.Obj):.2f}")


Optimization Status: ok
Total Objective Cost: €29047.00


In [30]:
print("Optimal production and delivery plan")

for t in model.T:
    print(f"\n PERIOD {t} ")
    
    for f in model.F:
        if pe.value(model.w[f, t]) > 0.5:
            print(f"\n Factory: {f}")
            total_steel_used = 0
            
            for p in model.P1:
                if pe.value(model.x1[f, p, t]) > 0:
                    bars = int(round(pe.value(model.x1[f, p, t])))
                    total_steel_used += bars * weights_lb['1']
                    print(f" • Cut {bars}x 9m bars with pattern {patterns_1[p]}")
                    
            for p in model.P2:
                if pe.value(model.x2[f, p, t]) > 0:
                    bars = int(round(pe.value(model.x2[f, p, t])))
                    total_steel_used += bars * weights_lb['2']
                    print(f"    • Cut {bars}x 12m bars with pattern {patterns_2[p]}")
                    
            print(f" Capacity Used: {total_steel_used:.2f} / {steel_capacity[f]} tonnes")
            
            print("\n Delivery Route")
            curr_node = f
            route = [curr_node]
            loaded_weight = 0
            
            while True:
                next_node = None
                for j in model.N:
                    if curr_node != j and pe.value(model.z[f, curr_node, j, t]) > 0.5:
                        next_node = j
                        break
                
                if next_node is None or next_node == f:
                    route.append(f)
                    break
                
                route.append(next_node)
                if next_node in model.C:
                    loaded_weight += customer_weight[next_node, t]
                curr_node = next_node
                
            print(f" Route: {' ➔ '.join(route)}")
            print(f" Vehicle Load: {loaded_weight:.2f} / {vehicle_capacity} tonnes")

Optimal production and delivery plan

 PERIOD 1 

 Factory: IJmuiden
 • Cut 24x 9m bars with pattern {'A': 0, 'B': 0, 'C': 2}
 • Cut 23x 9m bars with pattern {'A': 0, 'B': 2, 'C': 0}
 • Cut 12x 9m bars with pattern {'A': 3, 'B': 0, 'C': 0}
    • Cut 5x 12m bars with pattern {'A': 2, 'B': 2, 'C': 0}
 Capacity Used: 11.84 / 12 tonnes

 Delivery Route
 Route: IJmuiden ➔ Dortmund ➔ Hagen ➔ Iserlohn ➔ Schwerte ➔ Boenen ➔ IJmuiden
 Vehicle Load: 9.18 / 10 tonnes

 Factory: Segal
 • Cut 17x 9m bars with pattern {'A': 0, 'B': 0, 'C': 2}
    • Cut 4x 12m bars with pattern {'A': 0, 'B': 2, 'C': 1}
    • Cut 17x 12m bars with pattern {'A': 2, 'B': 2, 'C': 0}
 Capacity Used: 8.11 / 10 tonnes

 Delivery Route
 Route: Segal ➔ Neuss ➔ Bochum ➔ Gelsenkirchen ➔ Segal
 Vehicle Load: 6.30 / 10 tonnes

 PERIOD 2 

 Factory: IJmuiden
 • Cut 10x 9m bars with pattern {'A': 0, 'B': 0, 'C': 2}
 • Cut 26x 9m bars with pattern {'A': 0, 'B': 2, 'C': 0}
 • Cut 30x 9m bars with pattern {'A': 2, 'B': 0, 'C': 1}
 Cap